In [2]:
import os
os.chdir('/Users/Raphael/INRIA/travail/anHALytics/') #Select your working directory
cwd = os.getcwd()
import codecs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
SIRENE=pd.read_json('/Users/Raphael/INRIA/travail/SIRENE/subset_SIRENE.json')
SIRENE=SIRENE[SIRENE['SIEGE']==1]
org_SIREN=pd.read_json('org_SIREN.json')


In [4]:
data=pd.merge(SIRENE,org_SIREN,on='SIREN',how='right')
data['Count']=1

In [5]:
data.shape

(739, 102)

In [6]:
summary=data.groupby(['APEN700','LIBAPEN'],as_index=False).Count.count()
summary=summary.append({'APEN700':'Total','Count':sum(summary['Count'])},ignore_index=True)
summary['Proportion']=summary['Count'].apply(lambda x:x/summary['Count'][summary['APEN700']=='Total']).round(3)
summary['Percentage']=summary['Count'].apply(lambda x:x*100/summary['Count'][summary['APEN700']=='Total']).round(1)
summary['Percentage']=summary['Percentage'].apply(str)+'%'
summary

,APEN700,LIBAPEN,Count,Proportion,Percentage
0,6201Z,Programmation informatique,206,0.279,27.9%
1,6202A,Conseil en systèmes et logiciels informatiques,279,0.378,37.8%
2,6202B,Tierce maintenance de systèmes et d'applicatio...,13,0.018,1.8%
3,6203Z,Gestion d'installations informatiques,8,0.011,1.1%
4,6209Z,Autres activités informatiques,18,0.024,2.4%
5,6311Z,"Traitement de données, hébergement et activité...",47,0.064,6.4%
6,6312Z,Portails Internet,17,0.023,2.3%
7,7211Z,Recherche-développement en biotechnologie,19,0.026,2.6%
8,7219Z,Recherche-développement : autres sciences phys...,68,0.092,9.2%
9,7220Z,Recherche-développement en sciences humaines e...,5,0.007,0.7%


In [7]:
import plotly.plotly as py
import plotly.graph_objs as go

In [24]:
fig={"data":[{
    "values":list(summary['Count'][:-1]),
    "labels":list(summary['LIBAPEN'][:-1]),
    "domain": {"x": [0, .48]},
    "text": list(summary['APEN700'][:-1]),
    "name":"Code APE",
    "hoverinfo":"text+percent+name",
    "type": "pie",
    "marker":dict(line=dict(color='#000000', width=0.9))
    }],
     "layout": {
         "title":"Répartition des affiliations en fonction de l'acitivité de l'entreprise"
     }
    }
py.iplot(fig, filename='Répartition des affiliations en fonction du code APE')

In [13]:
import urllib
import bs4
from urllib import request

from urllib.request import Request, urlopen

def gethtml(link):
    req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    return(bs4.BeautifulSoup(webpage, "lxml"))

html_var=gethtml('http://sirene.fr/sirene/public/static/liste-variables')


L={}
for item in html_var.find_all('a',title="Voir la définition"):
    L[item.get('href').split('/')[-1].upper()]=item.getText().replace('XL2',"").replace('L2','')

L

{'ACTISURF': '  Type de magasin',
 'ACTIVNAT': '  Nature de l’activité de l’établissement',
 'APEN700-REV2': '  Activité principale de l’entreprise',
 'APET700-REV2': '  Activité principale de l’établissement',
 'APRM': '  Activité principale au registre des métiers',
 'ARRONET': '  Arrondissement de localisation de l’établissement',
 'AUXILT': '  Caractère auxiliaire de l’activité de l’établissement',
 'CIVILITE': ' Civilité des entrepreneurs individuels',
 'CODPOS': '  Code postal',
 'COMET': '  Commune de localisation de l’établissement',
 'CTONET': '  Canton de localisation de l’établissement',
 'DAPEN': '  Année de validité de l’activité principale de l’entreprise',
 'DAPET': '  Année de validité de l’activité principale de l’établissement',
 'DATEMAJ': ' Date de traitement de la mise à jour ',
 'DATEVE': ' Date de l’événement',
 'DDEBACT': '  Date de début d’activité',
 'DEFEN': '  Année de validité de l’effectif salarié de l’entreprise',
 'DEFET': '  Année de validité de l’effec

In [14]:
reg_file=cwd+'/Liste Regions/reg2016.csv'

regions=pd.read_csv(reg_file,sep=';',encoding='ISO-8859-1',index_col='RPET',skiprows=[1,2,3,4,5])

import geocoder

gb=data[['RPET','SIREN']].groupby('RPET',as_index=False).agg('count')
gb.index=gb['RPET']
reg_gb=pd.concat([regions[['CHEFLIEU','NCCENR']],gb], axis=1, join='inner')

reg_gb['LONG']=''
reg_gb['LAT']=''

reg_gb

reg_gb.index=[i for i in range(reg_gb.shape[0])]

for i in range(reg_gb.shape[0]):
    coord=geocoder.google(reg_gb['CHEFLIEU'][i]+' France').latlng
    reg_gb['LONG'][i]=coord[1]
    reg_gb['LAT'][i]=coord[0]


/Users/Raphael/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Raphael/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [17]:
scale=sum(reg_gb['SIREN'])/5000
reg_gb['text']=reg_gb['NCCENR'].map(str)+' - '+reg_gb['SIREN'].map(str)+' entreprises'

city = [dict(
        type = 'scattergeo',
        locationmode = 'france',
        showframe = False,
        showcoastlines = True,
        showland = True,
        lon = reg_gb['LONG'],
        lat = reg_gb['LAT'],
        text = reg_gb['text'],
        marker = dict(
            size = reg_gb['SIREN']/scale,
            color = reg_gb['SIREN'],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ))]


layout = dict(
        title = 'Repartition géographique des entreprises de la base SIREN',
        showlegend = False,
        geo = dict(
            projection=dict( type='france' ),
            showland = True,
            scope='europe',
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )


fig = dict( data=city, layout=layout )
py.iplot(fig, validate=False, filename='Buble Map SIREN' )

In [18]:
reg_gb

,CHEFLIEU,NCCENR,RPET,SIREN,LONG,LAT,text
0,75056,Ile-de-France,11.0,349,2.35222,48.8566,Ile-de-France - 349 entreprises
1,45234,Centre-Val de Loire,24.0,18,1.90925,47.903,Centre-Val de Loire - 18 entreprises
2,21231,Bourgogne-Franche-Comt_,27.0,5,5.04148,47.322,Bourgogne-Franche-Comt_ - 5 entreprises
3,76540,Normandie,28.0,7,0.520928,49.7615,Normandie - 7 entreprises
4,59350,Nord-Pas-de-Calais-Picardie,32.0,24,3.05015,50.6577,Nord-Pas-de-Calais-Picardie - 24 entreprises
5,67482,Alsace-Champagne-Ardenne-Lorraine,44.0,22,7.75211,48.5734,Alsace-Champagne-Ardenne-Lorraine - 22 entrepr...
6,44109,Pays de la Loire,52.0,18,-1.55362,47.2184,Pays de la Loire - 18 entreprises
7,35238,Bretagne,53.0,28,-1.67779,48.1173,Bretagne - 28 entreprises
8,33063,Aquitaine-Limousin-Poitou-Charentes,75.0,31,-0.57918,44.8378,Aquitaine-Limousin-Poitou-Charentes - 31 entre...
9,31555,Languedoc-Roussillon-Midi-Pyr_n_es,76.0,50,1.44421,43.6047,Languedoc-Roussillon-Midi-Pyr_n_es - 50 entrep...
